# Querying Astronomical Databases for Redshift Data


This notebook demonstrates how to connect to public astronomical databases, such as the Sloan Digital Sky Survey (SDSS) and MAST (Mikulski Archive for Space Telescopes), to retrieve redshift (*z*) data. 

We will perform the following steps:
1. Query the SDSS database using SQL through the `astroquery` library.
2. Query JWST data from the MAST archive using `astroquery`.
3. Save results in CSV format for further analysis.
    

In [2]:
# Install necessary libraries
!pip install astroquery astropy

Defaulting to user installation because normal site-packages is not writeable


DEPRECATION: Loading egg at c:\program files\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


## Querying SDSS for Redshift Data

In [2]:

from astroquery.sdss import SDSS
from astropy.table import Table

# SQL query to retrieve redshift data
query = """
-- Consulta SQL para recuperar datos y calcular distancias sin usar el corrimiento al rojo
WITH DistanceData AS (
    SELECT TOP 100000
        p.objID, p.ra, p.dec, p.u, p.g, p.r, p.i, p.z,
        -- Añadir magnitud aparente (m) y magnitud absoluta (M)
        p.r AS m,  -- Suponiendo que la magnitud aparente es la magnitud en la banda r
        -19.3 AS M  -- Suponiendo una magnitud absoluta típica para supernovas tipo Ia
    FROM PhotoObj AS p
    ORDER BY p.r DESC
)
SELECT *,
    -- Calcular la distancia (D) en parsecs usando la fórmula de la distancia de módulo
    POWER(10, (m - M + 5) / 5) AS D
FROM DistanceData;
"""

try:
    # Execute the query
    results = SDSS.query_sql(query)
    if results:
        # Convert results to CSV
        table = Table(results)
        table.write("sdss_query_results.csv", format="csv", overwrite=True)
        print("SDSS data saved to 'sdss_query_results.csv'")
    else:
        print("No data found in SDSS query.")
except Exception as e:
    print(f"Error querying SDSS: {e}")
    

Error querying SDSS: Some errors were detected !
    Line #3 (got 12 columns instead of 1)


## Querying JWST Data from MAST

In [4]:

from astroquery.mast import Mast

# Define query parameters for JWST data
query = {
    "obs_collection": "JWST",
    "filters": ["F200W"],  # Example filter
    "dataRights": "PUBLIC"  # Public data only
}

try:
    # Execute the query
    results = Mast.service_request_async("Mast.Caom.Filtered", query).to_table()
    # Save results as CSV
    results.write("jwst_data.csv", format="csv", overwrite=True)
    print("JWST data saved to 'jwst_data.csv'")
except Exception as e:
    print(f"Error querying MAST: {e}")
    

Error querying MAST: 'list' object has no attribute 'to_table'


## Notes and Further Steps


- Both queries use the `astroquery` library to interact with SDSS and MAST databases.
- The SDSS query retrieves photometric and spectroscopic data with redshifts.
- The MAST query focuses on JWST observations with specified filters and public data.
- You can adapt the SQL or query parameters to match your specific scientific needs.
- Results are saved in CSV format for further analysis.

### Dependencies
Ensure that the `astroquery` and `astropy` libraries are installed. Use `pip install astroquery astropy` if they are missing.

### Expanding the Analysis
- Include additional fields like distances or times since the Big Bang by incorporating cosmological calculations.
- Use other public datasets like ESA Sky or Vizier for complementary data.
    